### MCP Deep dive

Creating and using own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [ ]:
# Importing required libraries

from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from dotenv import load_dotenv
from IPython.display import Markdown, display

# importing python class from accounts.py
from accounts import Account

In [ ]:
load_dotenv(override=True)

In [ ]:
account = Account.get("Ian")
account

In [ ]:
account.buy_shares("AMZN", 4, "The bookstore looks promising in the future.")

In [ ]:
account.report()

In [ ]:
account.list_transactions()

#### Writing an MCP server and using it directly!

In [ ]:
# Using accounts server as an MCP server

params = {
    "command": "uv", 
    "args": ["run", "accounts_server.py"]
    }

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

In [ ]:
from agents.extensions.models.litellm_model import LitellmModel
import os

In [ ]:
api_key = os.getenv("AMAZON_BEDROCK_API_KEY")

In [ ]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ian and my account is under the name Ian. What's my balance and my holdings?"
model_name = "us.amazon.nova-micro-v1:0"
model=LitellmModel(model=model_name, api_key=api_key)

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(
        name="account_manager",
        model=model,
        instructions=instructions,
        mcp_servers=[mcp_server]
    )

    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

#### Building our own MCP Client

In [ ]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

In [ ]:
request = "My name is Ian and my account is under the name Ian. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(
        name="account_manager", 
        instructions=instructions, 
        model=model, 
        tools=openai_tools
    )
    
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
context = await read_accounts_resource("Ian")
print(context)

In [ ]:
Account.get("Ian").report()